In [ ]:
# This notebook will clean web statistics data by:
#
# 1) read a csv file containing web statistics from a hemispheres.com web site for an art gallery company
# 2) profile the dataset
# 3) identify important missing data
# 4) access a third party API to supply missing location data
# 5) update the web statistics data with the API's location data
# 6) compare the before and after missing data is supplied to prove the effect

In [ ]:
import pandas as pd
import pandas_profiling # a very useful and powerful package - https://github.com/ydataai/pandas-profiling
import numpy as np

In [ ]:
web_stats_df = pd.read_csv("statz_CSV_titles.csv")

In [ ]:
np.random.seed(0)

In [ ]:
# Gather some information about the web stats file and save it for a later comparison

In [ ]:
web_stats_profile_report = web_stats_df.profile_report()

In [ ]:
# Print some sample rows from the web_stats dataframe to better see its file contents

In [ ]:
web_stats_df.sample(5)

In [ ]:
# calculate and print a count of all missing values. This will serve as control counts to compare with our work below

In [ ]:
missing_values_count = web_stats_df.isnull().sum()
missing_values_count[0:24]

In [ ]:
#Calculate total cells of the web status data frame, sum up the missing values and report the percentage

In [ ]:
web_stats_total_cells = np.product(web_stats_df.shape)
web_stats_total_missing_values = missing_values_count.sum()
(web_stats_total_missing_values/web_stats_total_cells) * 100

In [ ]:
# Begin the process of filling in some of the most important missing data using data returned from an API call 
# to the IP-API public site

# Set up request to the ip-api.com API

## We are only doing 4 for demonstration purposes because more might incur costs.
##
## If we were going to fix all the missing values in web_stats_df we would select all of the web stat rows with 
## missing data 
## with something like this: 
##
##    web_stats_null_rows_df = web_stats_df[web_stats_df['CITY'].isnull() | web_stats_df['REGION'].isnull() 
##     | web_stats_df['REGIONCODE'].isnull() | web_stats_df['COUNTRY'].isnull()]
##
## and send all of the IP values in the API request instead of only four. 

In [ ]:
import time
import requests
import json
from pandas import json_normalize

url = "http://ip-api.com/batch?fields=status,message,country,region,regionName,city,zip,query"

#Build payload for request. 
payload = "[\"154.54.249.207\", \"91.198.174.192\", \"141.98.81.23\", \"66.249.75.38\"]"

#Create content-type header for request. We're sending the request payload as plain text
headers = {
  'Content-Type': 'text/plain'
}

In [ ]:
#Send the header and payload using a POST request, check the response status code (200 = OK) 
# and, if there's an exception, print the raw json response payload since it's small so we can see what we're 
# dealing with

In [ ]:
try:
    response = requests.request("POST", url, headers=headers, data=payload)
    if response.status_code != 200:
        raise Exception(f'Request failed with status code {response.status_code}')  

except Exception as e:
    print('http status code = ' + str(response.status_code))
    print('response json payload = ' + response.text)
    print('Request failed:', e)

In [ ]:
# Parse the json response payload text into a pandas dictionary, create a pandas dataframe from the dictionary 
# and print it

In [ ]:
locations = json.loads(response.text)

location_df = pd.DataFrame(locations)

print(location_df)

In [ ]:
# rename region of the location_df columns to the same name of the web_stats_df. 
# We have to do this in 2 steps because there's a name colision with region.
location_df.rename(columns={"region": "REGIONCODE"}, inplace = True)

# rename some of the location_df columns to the same name of the web_stats_df columns for easier handling
location_df.rename(columns={"country": "COUNTRY", "regionName": "REGION", "city": "CITY", "zip": "ZIP", "query": "IP"}, 
                   inplace = True)

print(location_df)

In [ ]:
# The code first defines a list called missing_cols that contains the column names 'COUNTRY', 'REGION', and 'CITY'.

# The code then uses a for loop to iterate through the rows of web_stats_df. 
# For each row, the code checks if any of the columns in the missing_cols list contain a null value. 
# If any of the columns do contain a null value, the code uses the IP value in the current row of web_stats_df to 
# find the corresponding row in the location_df DataFrame.

# Once the matching row is found, the code then updates the null values in the web_stats_df DataFrame with the data 
# from the matching row in the location_df DataFrame. 
# This process is repeated for each row in the web_stats_df DataFrame until all of the missing columns have been filled in.

In [ ]:
# Define the columns that need to be filled
missing_cols = ['COUNTRY', 'REGIONCODE', 'REGION', 'CITY']
matched = 0
replaced = 0

# Loop through the rows in the first DataFrame
for index, row in web_stats_df.iterrows():
    
    # Check if any of the missing columns have a null value
    if row[missing_cols].isnull().any():
        
        # Get the corresponding row from the location_df dataFrame based on the IP value
        match = location_df[location_df['IP'] == row['IP']]
        if not match.empty:
            print("found IP match")
            matched = matched + 1
            # Update the missing columns with the data from the matching row
            for col in missing_cols:
                if pd.isnull(row[col]):
                    web_stats_df.loc[index, col] = match.iloc[0][col]
                    print("replaced : " + web_stats_df[index, col])
                    replaced = replaced + 1
                else:
                    print("not replaced : " + web_stats_df[index, col])
                    
print("Matched = " + str(matched))
print("Replaced = " + str(replaced))

In [ ]:
missing_values_count = web_stats_df.isnull().sum()
missing_values_count[0:24]

In [ ]:
updated_web_stats_profile_report = web_stats_df.profile_report()

In [ ]:
# Compare the original web_stats_profile_report that we created at the beginning to this one
comparison_report = updated_web_stats_profile_report.compare(web_stats_profile_report)

In [ ]:
comparison_report.to_file("comparison.html")